# Opérations sur une base de données distante avec pandas et SQLAlchemy

Le but de ce notebook et d'expérimenter la manipulation de données sur une base distante (hébergée par Scaleway en mode serverless) à l'aide de `sqlachemy` (pour gérer la connexion) et `pandas` pour faire les opérations.

Les étapes prévisionnelles : 

- se connecter à la base de données avec SQLAlchemy
- créer une table et écrire dedans avec pandas
- lire le contenu de la table

## Pré-requis et constantes

Je commence par charger mes variables d'environnement (dans un fichier `.env` à la racine de mon répertoire). Ces variables comprennent en particulier mes identifiants, ainsi que les autres informations de connexion à la base de données. 

Pour cela j'ai besoin du module `dotenv`, à installer sous le nom de `python-dotenv`.

In [1]:
import os
from pathlib import Path

from dotenv import load_dotenv

In [2]:
load_dotenv("../.env")

True

In [3]:
PGUSER = os.getenv("PGUSER")
PGPASSWORD = os.getenv("PGPASSWORD")
PGHOST = os.getenv("PGHOST")
PGPORT = os.getenv("PGPORT")
PGDATABASE = os.getenv("PGDATABASE")

Je définis aussi le chemin du répertoire où je stockerai mes données :

In [4]:
PATH_TO_DATA = Path("../data/")

## Se connecter à la base de données avec SQLAlchemy

On se connecte à la BDD en utilisant un object `engine` de `sqlalchemy`, auquel on passe notre chaîne de caractères de connexion. 

La partie `"postgresql+psycopg2"` définit le dialecte SQL utilisé ainsi que le pilote SQL. `psycopg2` est déjà le pilote par défaut utilisé par `sqlalchemy`, mais je voulais faire les choses de manière explicite.
Dans tous les cas, il faudra installer le pilote et l'ajouter dans les dépendances.

In [5]:
from sqlalchemy import create_engine

In [6]:
engine = create_engine(
    f"postgresql+psycopg2://{PGUSER}:{PGPASSWORD}@{PGHOST}:{PGPORT}/{PGDATABASE}?sslmode=require&options=databaseid%3De5e14878-0ed7-4bc2-a80f-8c05ef81fac0",
)

In [7]:
engine.connect()

Le code ne renvoie pas d'erreur, la connexion s'est donc bien établie.

## Écrire des données avec pandas

Je vais écrire les données du poids total en aquaculture par année et par pays dans la BDD. J'en profite pour faire une table avec les correspondances des noms des pays (je me contente des langues latines parce que je ne sais pas exactement comment fonctionne l'encodage des caractères chinois, arabes et russes)

NOTE : Je suggère de ne pas utiliser `pandas` pour insérer des données dans la base de données en production : a priori, on ne peut pas spécifier le schéma de la table créée, donc pas de vérification des données, pas d'indexes, pas de clés étrangères. Mais pour une démo vite fait, ça passe.

In [8]:
import pandas as pd

In [9]:
df = pd.read_excel(PATH_TO_DATA / "atlantic_salmon-aquaculture_tonnes_live_weight_by_country_by_year.xlsx")

In [10]:
df_aquaculture = (
    df[["Year", "ISO2 Code", "Tonnes - live weight"]]
    .rename(columns={"Year": "year", "ISO2 Code": "iso2", "Tonnes - live weight": "tonnes"})
)

In [11]:
df_aquaculture

,year,iso2,tonnes
0,1950,GB,0.000
1,1950,SE,0.000
2,1950,NO,0.000
3,1950,KP,0.000
4,1950,IE,0.000
...,...,...,...
895,2021,FO,115650.000
896,2021,DK,1668.000
897,2021,CL,725280.000
898,2021,CA,120186.000


Pour insérer les données en tant que table dans notre base, on utilise la méthode `.to_sql()` de notre dataframe. Note : si la table existe déjà, on peut spécifier si on souhaite y concaténer les données, remplacer les données actuelles, ou renvoyer une erreur. Ici, j'ai choisi de remplacer les données actuelles.

In [20]:
df_aquaculture.to_sql(
    name="aquaculture_weight_by_country",
    con=engine,
    if_exists="replace",
    index=False)

900

Pour l'exemple, on fait la même chose avec les noms de pays :

In [21]:
df_country_names = (
    df[[
        'ISO2 Code',
        'Country Name En',
        'Country Name Es',
        'Country Name Fr',
    ]]
    .rename(columns={
        'ISO2 Code': 'iso2',
        'Country Name En': 'english',
        'Country Name Es': 'spanish',
        'Country Name Fr': "french",
    })
    .drop_duplicates()
    .reset_index(drop=True)
)

In [22]:
df_country_names

,iso2,english,spanish,french
0,GB,United Kingdom,Reino Unido,Royaume-Uni
1,SE,Sweden,Suecia,Suède
2,NO,Norway,Noruega,Norvège
3,KP,"Korea, Dem. People's Rep",Rep. Pop. Dem. de Corea,Rép. pop. dém. de Corée
4,IE,Ireland,Irlanda,Irlande
5,FI,Finland,Finlandia,Finlande
6,CA,Canada,Canadá,Canada
7,IS,Iceland,Islandia,Islande
8,CL,Chile,Chile,Chili
9,FO,Faroe Islands,Islas Feroe,Îles Féroé


In [23]:
df_country_names.to_sql(
    name='country_names',
    con=engine,
    if_exists='replace',
    index=False
)

22

## Récupérer les données avec pandas

On va récupérer les données qu'on a écrites dans la base avec pandas. La méthode `.read_sql()` permet de spécifier une requête SQL a exécuter, ou le nom d'une table à lire. Démonstration.

In [24]:
df_aqua_1 = pd.read_sql(
    sql='aquaculture_weight_by_country',
    con=engine
)
df_aqua_2 = pd.read_sql(
    sql="SELECT * FROM aquaculture_weight_by_country",
    con=engine
)

In [28]:
df_aqua_1.equals(df_aqua_2)

True